先把之前写的那个丑丑的debug搬过来

In [4]:
(defmacro my-debug
  [expr]
  (list 'let ['result expr]
        (list 'println (list 'quote expr)) "=" 'result)
        'result))

(var user/my-debug)

用backquote试试

In [5]:
(defmacro my-new-debug
  [expr]
  `(let [result expr]
     (println (quote expr) "=" result)
     result))

(var user/my-new-debug)

In [6]:
(my-new-debug (+ 1 2))

class clojure.lang.Compiler$CompilerException: CompilerException java.lang.RuntimeException: Can't let qualified name: user/result, compiling:(NO_SOURCE_PATH:1:1) 


呃。。。。啥？木有result？发生了什么？

其实Clojure不能让你let user/result这个qualified name。 这里result不做点特殊处理的话，不能被当做let binding的对象。特殊处理是什么呢？就是#。为什么不能直接let呢？Clojure是为了防止variable capture这个问题。如果不用这个#的话，如果外面有一个名叫result的对象，在宏内就无法区分你是在指外面的对象还是想指宏这个了。

如果我们一定要用result来bind咋办？

In [7]:
(defmacro my-new-debug-bad
  [expr]
  (list 'let ['result expr]
    (list 'println (list 'quote expr) "=" 'result)
    'result))

(var user/my-new-debug-bad)

In [8]:
(def result 7)

(var user/result)

In [9]:
(my-new-debug-bad (+ 1 2))

(+ 1 2) = 3


3

呐，我们原来定义过的result就被“遮掉了”，要是我们的原意本来就是想要外面那个result咋办？这个就叫variable capture。。那我们老老实实用#吧

In [10]:
(defmacro my-new-debug
  [expr]
  `(let [result# expr]
     (println (quote expr) "=" result#)
     result))

(var user/my-new-debug)

In [11]:
(my-new-debug (+ 1 2))

class clojure.lang.Compiler$CompilerException: CompilerException java.lang.RuntimeException: No such var: user/expr, compiling:(NO_SOURCE_PATH:1:1) 


你们肯定在想，怎么还没写好啊！expr又出啥事儿了？！咱们用macroexpand-1来看看。。

In [12]:
(macroexpand-1 '(my-new-debug (+ 1 2)))

(clojure.core/let [result__42__auto__ user/expr] (clojure.core/println (quote user/expr) "=" result__42__auto__) user/result)

这里expr没有被evaluate。。因为我们没有unquote！

In [13]:
(defmacro my-new-debug
  [expr]
  `(let [result# ~expr]
     (println (quote ~expr) "=" result#)
     result#))

(var user/my-new-debug)

In [14]:
(my-new-debug (+ 1 2))

(+ 1 2) = 3


3

耶！我们的debug宏写完辣！然而。。。